# Import Customized modules and Create ImageJ Object

In [ ]:
import os
import sys
import re
from pathlib import Path

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.data.ij.zfij import ZFIJ
from modules.data.lif import LIFNameChecker, scan_lifs_under_dir
from modules.shared.pathnavigator import PathNavigator

from modules.shared.logger import init_logger
from modules.shared.utils import get_repo_root, load_config

from modules.assert_fn import assert_lifname_split_in_4_part

Initialize `Customized Object`

In [ ]:
logger = init_logger(r"Check Lif Name")
display_kwargs = {
    "display_on_CLI": True,
    "logger": logger
}

lif_name_checker = LIFNameChecker()
path_navigator = PathNavigator()
repo_root = get_repo_root(**display_kwargs)

Initialize `Fiji` ( This will change the working directory to where the `JVM` exists. )

In [ ]:
zfij = ZFIJ()

Load `0.1.check_lif_name.toml`

In [ ]:
config = load_config("0.1.check_lif_name.toml", **display_kwargs)
nasdl_type    = config["data_nasdl"]["type"]
nasdl_batches = config["data_nasdl"]["batches"]

# Main Process

Get `paths`

In [ ]:
lif_scan_root = path_navigator.raw_data.get_lif_scan_root(config, **display_kwargs)

Run

In [ ]:
lif_path_list = scan_lifs_under_dir(lif_scan_root, nasdl_batches, logger=logger)


for i, lif_path in enumerate(lif_path_list):
    
    logger.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    logger.info(f'LIF_FILE : {lif_path}')
    
    
    """ Normalize LIF name """
    lif_name = lif_path.split(os.sep)[-1].split(".")[0]
    lif_name_list = re.split(" |_|-", lif_name)
    assert_lifname_split_in_4_part(lif_name_list, lif_name)
    lif_name = "_".join(lif_name_list)
    
    """ Get number of images in LIF file """
    zfij.imageReader.setId(lif_path)
    series_cnt = zfij.imageReader.getSeriesCount()
    
    
    for idx in range(series_cnt):
        
        series_num = idx+1
        
        """ Read image """
        zfij.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{series_num}")
        img = zfij.ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        img.hide()
        
        """ Get image name """
        image_name = str(img.getProp("Image name"))
        
        
        """ BrightField_RAW """
        if nasdl_type == "BrightField_RAW":
            if "Before_20221109" in lif_path:
                lif_name_checker.check_image_name(image_name, "old_bf")
            else: 
                lif_name_checker.check_image_name(image_name, "new_bf")
        
        """ PalmSkin_RAW """
        if nasdl_type == "PalmSkin_RAW":
            if "Before_20221109" in lif_path:
                lif_name_checker.check_image_name(image_name, "old_rgb")
            else: 
                lif_name_checker.check_image_name(image_name, "new_rgb")
        
        
        """ Concat LIF and image name """
        comb_name = f"{lif_name} - {image_name}"
        
        """ Get image dimension """
        img_dimensions = img.getDimensions()
        logger.info(f"series {series_num:{len(str(series_cnt))}}/{series_cnt} : '{comb_name}' , "
                    f"Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        
        """ Print ERRORs after checking 'image name' """
        try:
            logger.info(f"       ##### {lif_name_checker.check_dict['failed message']}")
        except KeyError:
            pass
    
    
    logger.info("\n")


logger.info(" -- finished -- ")